# Amazon FinSpace Managed kdb Insights: TAQ

Query the TAQ table

## Algoseek LLC Data
Trade and Quote data has been provided by [AlgoSeek LLC](https://www.algoseek.com/), you can learn more about their data offerings from their home page.


In [1]:
import os
import subprocess
import boto3
import json
import datetime

import pandas as pd
import pykx as kx

pd.set_option('display.max_rows', 10)
kx.q.system.display_size = [50, 1000]

from managed_kx import *
from env import *

# ----------------------------------------------------------------
CLUSTER_NAME="demo_csv_cluster"
# ----------------------------------------------------------------

In [2]:
# Using credentials and create service client
session = boto3.Session()

# create finspace client
client = session.client(service_name='finspace')

# Connect to Cluster with PyKX

In [3]:
hdb = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

# Tables and counts

In [4]:
# be sure the database is loaded
hdb('.Q.l `$.aws.akdbp,"/",.aws.akdb')

# delete the res table
hdb("delete r from `.")

pykx.SymbolAtom(pykx.q('`.'))

In [5]:
# inventory of tables in the database and rows in each
print(80*'=')
print("All Tables and Counts")
display( hdb("tables[]!count each value each tables[]") )


All Tables and Counts


,
,
res,17211
taq,15144970


# Table Summary
- Rows   
- Schema
- Rows per Date
- Sample 

In [6]:
tables = hdb("tables[]").py()

# For each table: schema, and samples and counts
for t in tables:
    c=hdb(f"count {t}").py()

    print(80*'=')
    print (f'Table: {t}: {c:,}')
    print(80*'-')
    display( hdb(f"meta {t}") )
#    display( hdb(f"select rows:count i by Ticker from {t} where date=max date ") )
    display( hdb(f"select rows:count i by date from {t}") )
    display( hdb(f"select from {t} where date = max date, i<3") )

Table: res: 17,211
--------------------------------------------------------------------------------


,t,f,a
c,,,
date,"""d""",,
Ticker,"""s""",,p
EventType,"""s""",,
minute,"""u""",,
Events,"""j""",,
Avg_Price,"""f""",,
Volume,"""j""",,
Max_Price,"""f""",,
Min_Price,"""f""",,


,rows
date,
2021.01.05,17211


pykx.KeyedTable(pykx.q('
date       Ticker EventType minute| Events Avg_Price Volume Max_Price Min_Price First_Price Last_Price
----------------------------------| ------------------------------------------------------------------
2021.01.05 AMZN   QUOTE ASK 04:00 | 13     2708.931  1300   3202      0         0           3201      
2021.01.05 AMZN   QUOTE ASK 04:01 | 21     2895.349  2700   3200.29   0         3200.29     3200      
2021.01.05 AMZN   QUOTE ASK 04:02 | 28     1601.335  1600   3221.9    0         3200        3213.49   
'))

Table: taq: 15,144,970
--------------------------------------------------------------------------------


,t,f,a
c,,,
date,"""d""",,
Ticker,"""s""",,p
Timestamp,"""n""",,
EventType,"""s""",,
Price,"""f""",,
Quantity,"""j""",,
Exchange,"""s""",,
Conditions,"""s""",,
FileName,"""s""",,


,rows
date,
2021.01.04,8970726
2021.01.05,6174244


,date,Ticker,Timestamp,EventType,Price,Quantity,Exchange,Conditions,FileName,FileExtension
,,,,,,,,,,
0,2021.01.05,AMZN,0D04:00:00.021680902,TRADE,3190.01,63,ARCA,80000401,AMZN,gz
1,2021.01.05,AMZN,0D04:00:00.023083159,QUOTE BID,2000f,400,ARCA,00000001,AMZN,gz
2,2021.01.05,AMZN,0D04:00:00.023083159,QUOTE ASK,0f,0,ARCA,00000001,AMZN,gz


# Summarize Events
Number of events, average price, total volume grouped by date, Ticker, and EventType

**Note:** SPY is an ETF, no asks

In [7]:
# Summary of a day's events for a few tickers
q="""select Events:count Price, Avg_Price:avg Price, Volume:sum Quantity 
    by date, Ticker, EventType 
    from taq
    where date = max date, any Ticker in/:(`AMZN;`SPY)"""

res = hdb(q).pd()
display( res )

Events    Avg_Price    Volume
date       Ticker EventType                                  
2021-01-05 AMZN   QUOTE ASK     615762  3191.515952  75072400
                  QUOTE ASK NB  158824  3203.824669  20565900
                  QUOTE BID     616379  3150.847312  74631400
                  QUOTE BID NB  158826  3201.910941  19186200
                  TRADE         129553  3202.309602   1779429
                  TRADE NB        6077  3203.388657   1091928

# Calculate 1 Minute Bars

In [8]:
# Aggregate minute bars for a day, all tickers and event types
q="""select 
        Events: count Price, 
        Avg_Price: avg Price, 
        Volume: sum Quantity, 
        First_Price: first Price, Last_Price: last Price,
        Min_Price: min Price, Max_Price: max Price
    by Ticker, EventType, date, 1 xbar Timestamp.minute 
    from taq 
    where date=max date, Ticker=`AMZN, EventType=`TRADE"""

display( hdb(q) )

# get row count for max date
c=hdb("count select i from taq where date = max date").py()
print( f"Rows In Date: {c:,}" )

pykx.KeyedTable(pykx.q('
Ticker EventType date       minute| Events Avg_Price Volume First_Price Last_Price Min_Price Max_Price
----------------------------------| ------------------------------------------------------------------
AMZN   TRADE     2021.01.05 04:00 | 9      3191.7    179    3190.01     3192.37    3190.01   3195     
AMZN   TRADE     2021.01.05 04:01 | 8      3189.274  214    3190.01     3189.99    3187      3190.01  
AMZN   TRADE     2021.01.05 04:02 | 7      3191.156  52     3188.01     3194.59    3188      3194.59  
AMZN   TRADE     2021.01.05 04:03 | 3      3192.72   51     3190        3194.59    3190      3194.59  
AMZN   TRADE     2021.01.05 04:04 | 7      3190.477  201    3191.19     3190       3190      3191.35  
AMZN   TRADE     2021.01.05 04:05 | 3      3190.017  15     3190.05     3190       3190      3190.05  
AMZN   TRADE     2021.01.05 04:06 | 2      3191.305  13     3190.61     3192       3190.61   3192     
AMZN   TRADE     2021.01.05 04:08 | 1      3190.92   16     3190.92     3190.92    3190.92   3190.92  
AMZN   TRADE     2021.01.05 04:09 | 6      3190.802  26     3190.96     3190       3190      3190.97  
AMZN   TRADE     2021.01.05 04:10 | 1      3190      1      3190        3190       3190      3190     
AMZN   TRADE     2021.01.05 04:11 | 1      3190.22   1      3190.22     3190.22    3190.22   3190.22  
AMZN   TRADE     2021.01.05 04:12 | 1      3190.04   1      3190.04     3190.04    3190.04   3190.04  
AMZN   TRADE     2021.01.05 04:13 | 2      3190.445  2      3190.21     3190.68    3190.21   3190.68  
AMZN   TRADE     2021.01.05 04:15 | 4      3190.887  7      3190.78     3190.7     3190.7    3191.07  
AMZN   TRADE     2021.01.05 04:16 | 3      3191.66   4      3191.16     3191.91    3191.16   3191.91  
AMZN   TRADE     2021.01.05 04:17 | 4      3191.972  11     3191.89     3192       3191.89   3192     
AMZN   TRADE     2021.01.05 04:18 | 1      3192      4      3192        3192       3192      3192     
AMZN   TRADE     2021.01.05 04:19 | 3      3192.997  30     3192.99     3193       3192.99   3193     
AMZN   TRADE     2021.01.05 04:20 | 1      3193      1      3193        3193       3193      3193     
AMZN   TRADE     2021.01.05 04:21 | 2      3192.9    4      3192.9      3192.9     3192.9    3192.9   
AMZN   TRADE     2021.01.05 04:24 | 7      3190.684  102    3192.79     3190       3190      3192.79  
AMZN   TRADE     2021.01.05 04:25 | 5      3190.392  18     3190.45     3190       3190      3190.85  
AMZN   TRADE     2021.01.05 04:26 | 4      3191.65   42     3191.77     3190.85    3190.85   3192     
AMZN   TRADE     2021.01.05 04:29 | 8      3189.86   234    3190.27     3189.98    3189.51   3190.27  
AMZN   TRADE     2021.01.05 04:30 | 4      3188.265  14     3188.57     3188       3188      3188.57  
AMZN   TRADE     2021.01.05 04:35 | 3      3190.543  7      3190.37     3190.63    3190.37   3190.63  
AMZN   TRADE     2021.01.05 04:36 | 1      3190.63   9      3190.63     3190.63    3190.63   3190.63  
AMZN   TRADE     2021.01.05 04:39 | 3      3190.81   5      3190.71     3190.34    3190.34   3191.38  
AMZN   TRADE     2021.01.05 04:42 | 1      3191.72   2      3191.72     3191.72    3191.72   3191.72  
AMZN   TRADE     2021.01.05 04:45 | 11     3193.024  49     3190.92     3193.09    3190.92   3194     
AMZN   TRADE     2021.01.05 04:46 | 1      3194.73   3      3194.73     3194.73    3194.73   3194.73  
AMZN   TRADE     2021.01.05 04:51 | 1      3194.65   1      3194.65     3194.65    3194.65   3194.65  
AMZN   TRADE     2021.01.05 04:52 | 1      3194.43   1      3194.43     3194.43    3194.43   3194.43  
AMZN   TRADE     2021.01.05 04:55 | 1      3193.09   1      3193.09     3193.09    3193.09   3193.09  
AMZN   TRADE     2021.01.05 04:57 | 1      3191      5      3191        3191       3191      3191     
AMZN   TRADE     2021.01.05 04:58 | 14     3189.894  143    3190.45     3188.5     3188.5    3190.56  
AMZN   TRADE     2021.01.05 04:59 | 11     3188.322  169    3

Rows In Date: 6,174,244


# Rows Per Ticker

In [9]:
# assign results to an on-cluster variable
hdb(f"res:select rows:count i by Ticker from taq where date = max date")

# sample results to a local variable
res = hdb('select [100] from res')

# display local (sampled) results
display(res)

c=hdb(f"count res").py()
print (f'Result Rows: {c:,}')

,rows
Ticker,
AMZN,1685421
FB,1720887
GOOG,1550965
NFLX,1216971


Result Rows: 4


# Calculate 1 Minute Bar High, Low, Open, Close (HLOC)
For one date

In [10]:
# get row count for max date
c=hdb("count select i from taq where date = max date").py()
print( f"Rows In Date: {c:,}" )

# Aggregate minute bars for a day, all tickers and event types
q="""res:select 
        Events: count Price, 
        Avg_Price: avg Price, 
        Volume: sum Quantity, 
        Max_Price: max Price, Min_Price: min Price, 
        First_Price: first Price, Last_Price: last Price
    by date, Ticker, EventType, 1 xbar Timestamp.minute 
    from taq 
    where date=max date"""

# execute query, sets results into a variable on-cluster
hdb(q)

# result rows
c=hdb(f"count res").py()
print (f'Result Rows: {c:,}')

# sample results to a local variable
res = hdb('select [100] from res')

# display local (sampled) results
display(res)

Rows In Date: 6,174,244
Result Rows: 17,211


pykx.KeyedTable(pykx.q('
date       Ticker EventType minute| Events Avg_Price Volume Max_Price Min_Price First_Price Last_Price
----------------------------------| ------------------------------------------------------------------
2021.01.05 AMZN   QUOTE ASK 04:00 | 13     2708.931  1300   3202      0         0           3201      
2021.01.05 AMZN   QUOTE ASK 04:01 | 21     2895.349  2700   3200.29   0         3200.29     3200      
2021.01.05 AMZN   QUOTE ASK 04:02 | 28     1601.335  1600   3221.9    0         3200        3213.49   
2021.01.05 AMZN   QUOTE ASK 04:03 | 22     3207.358  2200   3213.49   3200      3213.49     3200      
2021.01.05 AMZN   QUOTE ASK 04:04 | 43     3203.765  4300   3213.49   3200      3213.49     3200      
2021.01.05 AMZN   QUOTE ASK 04:05 | 37     3200.182  3800   3210.87   3194.85   3200        3200      
2021.01.05 AMZN   QUOTE ASK 04:06 | 40     3200.05   4200   3201      3200      3200        3200      
2021.01.05 AMZN   QUOTE ASK 04:07 | 32     3200.031  3300   3201      3200      3200        3200      
2021.01.05 AMZN   QUOTE ASK 04:08 | 34     3200      3400   3200      3200      3200        3200      
2021.01.05 AMZN   QUOTE ASK 04:09 | 34     3200.211  3400   3207.16   3200      3200        3200      
2021.01.05 AMZN   QUOTE ASK 04:10 | 34     3200.211  3400   3207.16   3200      3200        3200      
2021.01.05 AMZN   QUOTE ASK 04:11 | 37     3200.078  4200   3201      3200      3200        3200      
2021.01.05 AMZN   QUOTE ASK 04:12 | 26     3200.036  2800   3200.47   3200      3200.47     3200      
2021.01.05 AMZN   QUOTE ASK 04:13 | 39     3200.395  7100   3200.59   3200      3200.47     3200.59   
2021.01.05 AMZN   QUOTE ASK 04:14 | 33     3199.162  6500   3200.65   3198      3200.59     3198      
2021.01.05 AMZN   QUOTE ASK 04:15 | 32     3198.466  6000   3207.16   3198      3200        3198      
2021.01.05 AMZN   QUOTE ASK 04:16 | 38     3198.198  7200   3199.88   3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:17 | 33     3198.057  6500   3199.88   3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:18 | 15     3198      3000   3198      3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:19 | 25     3198.366  4900   3207.16   3198      3207.16     3198      
2021.01.05 AMZN   QUOTE ASK 04:20 | 30     3198.305  5900   3207.16   3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:21 | 24     3198      4800   3198      3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:22 | 41     3201.546  5400   3207.16   3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:23 | 31     3198      3100   3198      3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:24 | 37     3198.99   3700   3207.16   3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:25 | 16     3198      1600   3198      3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:26 | 22     3198      2200   3198      3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:27 | 40     3198.458  4000   3207.16   3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:28 | 29     3198      2900   3198      3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:29 | 42     3199.79   4200   3207.16   3198      3198        3207.16   
2021.01.05 AMZN   QUOTE ASK 04:30 | 43     3199.751  4300   3207.16   3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:31 | 46     3199.792  4600   3207.16   3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:32 | 55     3200.165  5500   3207.16   3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:33 | 51     3199.976  5100   3207.16   3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:34 | 53     3200.247  5300   3207.16   3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:35 | 25     3198      2500   3198      3198      3198        3198      
2021.01.05 AMZN   QUOTE ASK 04:36 | 21     3198.872  2100   3

# Calculate 1 Minute Bar High, Low, Open, Close (HLOC)
Over a list of dates

In [11]:
# all dates in table
res = hdb(f"select distinct date from taq").pd()

# list of dates
dlist = res['date'][:2] 

for d in dlist:
    hdb(".Q.gc[]")

    d = d.date()
    hdb['dt']=d

    # get row count for date
    c=hdb("count select i from taq where date = dt").py()

    print( f"Date {d}" )
    print( f"Rows: {c:,}" )

    # calculate and store results in res
    q=f"""res:select 
        Events: count Price, 
        Avg_Price: avg Price, 
        Volume: sum Quantity, 
        Max_Price: max Price, 
        Min_Price: min Price, 
        First_Price: first Price, 
        Last_Price: last Price
    by date, Ticker, EventType, 1 xbar Timestamp.minute 
    from taq 
    where date=dt"""

    # execute query, sets results into a variable on-cluster
    hdb(q)

    # result rows
    c=hdb('count res').py()
    print( f"Result Rows: {c:,}" )

    # sample results to a local variable
    res = hdb('select [10] from res')

    # display local (sampled) results
    display(res)

    print()

Date 2021-01-04
Rows: 8,970,726
Result Rows: 17,616


pykx.KeyedTable(pykx.q('
date       Ticker EventType minute| Events Avg_Price Volume Max_Price Min_Price First_Price Last_Price
----------------------------------| ------------------------------------------------------------------
2021.01.04 AMZN   QUOTE ASK 04:00 | 23     2573.147  2000   3288.36   0         0           3288.36   
2021.01.04 AMZN   QUOTE ASK 04:01 | 23     1858.638  1300   3288.36   0         3288.36     3288.36   
2021.01.04 AMZN   QUOTE ASK 04:02 | 27     2922.955  3100   3288.36   0         3288.36     3288.31   
2021.01.04 AMZN   QUOTE ASK 04:03 | 18     3288.31   3600   3288.31   3288.31   3288.31     3288.31   
2021.01.04 AMZN   QUOTE ASK 04:04 | 34     3308.243  4600   3460      3288.31   3288.31     3300      
2021.01.04 AMZN   QUOTE ASK 04:05 | 35     3343.765  3500   3380      3291.69   3380        3350      
2021.01.04 AMZN   QUOTE ASK 04:06 | 69     3323.91   6900   3360      3291.69   3291.69     3291.69   
2021.01.04 AMZN   QUOTE ASK 04:07 | 53     3295.514  5300   3300      3290.5    3300        3300      
2021.01.04 AMZN   QUOTE ASK 04:08 | 57     3295.387  5700   3300      3290.1    3290.5      3295.47   
2021.01.04 AMZN   QUOTE ASK 04:09 | 52     3293.987  5200   3300      3291.04   3291.04     3291.04   
'))


Date 2021-01-05
Rows: 6,174,244
Result Rows: 17,211


pykx.KeyedTable(pykx.q('
date       Ticker EventType minute| Events Avg_Price Volume Max_Price Min_Price First_Price Last_Price
----------------------------------| ------------------------------------------------------------------
2021.01.05 AMZN   QUOTE ASK 04:00 | 13     2708.931  1300   3202      0         0           3201      
2021.01.05 AMZN   QUOTE ASK 04:01 | 21     2895.349  2700   3200.29   0         3200.29     3200      
2021.01.05 AMZN   QUOTE ASK 04:02 | 28     1601.335  1600   3221.9    0         3200        3213.49   
2021.01.05 AMZN   QUOTE ASK 04:03 | 22     3207.358  2200   3213.49   3200      3213.49     3200      
2021.01.05 AMZN   QUOTE ASK 04:04 | 43     3203.765  4300   3213.49   3200      3213.49     3200      
2021.01.05 AMZN   QUOTE ASK 04:05 | 37     3200.182  3800   3210.87   3194.85   3200        3200      
2021.01.05 AMZN   QUOTE ASK 04:06 | 40     3200.05   4200   3201      3200      3200        3200      
2021.01.05 AMZN   QUOTE ASK 04:07 | 32     3200.031  3300   3201      3200      3200        3200      
2021.01.05 AMZN   QUOTE ASK 04:08 | 34     3200      3400   3200      3200      3200        3200      
2021.01.05 AMZN   QUOTE ASK 04:09 | 34     3200.211  3400   3207.16   3200      3200        3200      
'))

# SQL Query
Use PyKX's built in sql feature to use ANSI sql to query tabls.

## Reference
[KX SQL](https://code.kx.com/insights/1.11/core/sql.html)

In [12]:
hdb.sql("select * from taq limit 5")

,date,Ticker,Timestamp,EventType,Price,Quantity,Exchange,Conditions,FileName,FileExtension
,,,,,,,,,,
0,2021.01.04,AMZN,0D04:00:00.021805862,TRADE,3271.69,32,ARCA,80000401,AMZN,gz
1,2021.01.04,AMZN,0D04:00:00.023217706,QUOTE BID,2861f,200,ARCA,00000001,AMZN,gz
2,2021.01.04,AMZN,0D04:00:00.023217706,QUOTE ASK,0f,0,ARCA,00000001,AMZN,gz
3,2021.01.04,AMZN,0D04:00:00.023217706,QUOTE BID NB,2861f,200,ARCA,00000001,AMZN,gz
4,2021.01.04,AMZN,0D04:00:00.023471043,QUOTE BID NB,2861f,200,ARCA,00000001,AMZN,gz


# Q Queries
Using connection information from the service, and the q magic cell feature supplied by PyKX, execute q code directly on the remote cluster.

In [13]:
# get the connection string to the cluster
conn_str = get_kx_connection_string(client, 
                          environmentId=ENV_ID, clusterName=CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

# parse the connection string to components
host, port, username, password = parse_connection_string(conn_str)


In [14]:
%%q --host $host --port $port --user $username --pass $password

/ set console height and width
\c 25 5000

In [15]:
%%q --host $host --port $port --user $username --pass $password

/ workspace information
.Q.w[]

/ tables on the cluster
tables[]

used| 4752400
heap| 536870912
peak| 2751463424
wmax| 0
mmap| 0
mphy| 33172058112
syms| 4113
symw| 210729
`res`taq


# Show Schema of Table

In [16]:
%%q --host $host --port $port --user $username --pass $password

/ schema of a table
meta taq

c            | t f a
-------------| -----
date         | d    
Ticker       | s   p
Timestamp    | n    
EventType    | s    
Price        | f    
Quantity     | j    
Exchange     | s    
Conditions   | s    
FileName     | s    
FileExtension| s    


# Rows per Date 

In [17]:
%%q --host $host --port $port --user $username --pass $password

/ count rows by date
select counts:count i by date from taq

"Rows: ", {reverse "," sv 0N 3#reverse string x} count taq

date      | counts 
----------| -------
2021.01.04| 8970726
2021.01.05| 6174244
Rows: 15,144,970


# Aggregate Data

In [18]:
%%q --host $host --port $port --user $username --pass $password

/ define a function
sq:{x*x}

/ use function in a query
select avg_num:avg Price, t_cnt:count Price, vol:sum Quantity, avg_sq_price: sq[avg Price] 
    by date,Ticker 
    from taq
    where date=max date, Ticker in (`AAPL`AMZN`JPM`GS)                                          

date       Ticker| avg_num  t_cnt   vol       avg_sq_price
-----------------| ---------------------------------------
2021.01.05 AMZN  | 3179.655 1685421 192327257 1.011021e+07


In [19]:
%%q --host $host --port $port --user $username --pass $password

/ compute 5 minute high, low, open close
select 
        Price: avg Price, 
        Volume: avg Quantity, 
        First_Price: first Price, 
        Last_Price: last Price,
        Min_Price: min Price, 
        Max_Price: max Price, 
        Events: count Price 
    by Ticker,EventType,date,5 xbar Timestamp.minute 
    from taq 
    where date=max date,Ticker=`AMZN,EventType=`TRADE

Ticker EventType date       minute| Price    Volume   First_Price Last_Price Min_Price Max_Price Events
----------------------------------| -------------------------------------------------------------------
AMZN   TRADE     2021.01.05 04:00 | 3190.855 20.5     3190.01     3190       3187      3195      34    
AMZN   TRADE     2021.01.05 04:05 | 3190.699 5.833333 3190.05     3190       3190      3192      12    
AMZN   TRADE     2021.01.05 04:10 | 3190.23  1        3190        3190.68    3190      3190.68   5     
AMZN   TRADE     2021.01.05 04:15 | 3191.827 3.733333 3190.78     3193       3190.7    3193      15    
AMZN   TRADE     2021.01.05 04:20 | 3191.359 10.7     3193        3190       3190      3193      10    
AMZN   TRADE     2021.01.05 04:25 | 3190.438 17.29412 3190.45     3189.98    3189.51   3192      17    
AMZN   TRADE     2021.01.05 04:30 | 3188.265 3.5      3188.57     3188       3188      3188.57   4     
AMZN   TRADE     2021.01.05 04:35 | 3190.67  3        3190.37   

In [20]:
%%q --host $host --port $port --user $username --pass $password

/ average price by date, ticker, event type
select 
    AvgPrice:avg Price, 
    Events:count Price, 
    Volume:sum Quantity 
    by Ticker,EventType,date 
    from taq 
    where date=max date, Ticker in (`AMZN`JPM)


Ticker EventType    date      | AvgPrice Events Volume  
------------------------------| ------------------------
AMZN   QUOTE ASK    2021.01.05| 3191.516 615762 75072400
AMZN   QUOTE ASK NB 2021.01.05| 3203.825 158824 20565900
AMZN   QUOTE BID    2021.01.05| 3150.847 616379 74631400
AMZN   QUOTE BID NB 2021.01.05| 3201.911 158826 19186200
AMZN   TRADE        2021.01.05| 3202.31  129553 1779429 
AMZN   TRADE NB     2021.01.05| 3203.389 6077   1091928 


In [21]:
%%q --host $host --port $port --user $username --pass $password

/ average price by date, hour, ticker, event type
select 
    AvgPrice:avg Price, 
    Events:count Price, 
    Volume:sum Quantity 
    by Ticker,EventType,date,Timestamp.hh 
    from taq 
    where date=max date, Ticker in (`AMZN`JPM)
    

Ticker EventType    date       hh| AvgPrice Events Volume  
---------------------------------| ------------------------
AMZN   QUOTE ASK    2021.01.05 4 | 3171.53  1975   224300  
AMZN   QUOTE ASK    2021.01.05 5 | 3202.886 2151   227600  
AMZN   QUOTE ASK    2021.01.05 6 | 3192.84  1474   148400  
AMZN   QUOTE ASK    2021.01.05 7 | 3124.191 3998   1023900 
AMZN   QUOTE ASK    2021.01.05 8 | 2997.59  6601   765900  
AMZN   QUOTE ASK    2021.01.05 9 | 3187.085 78196  9725000 
AMZN   QUOTE ASK    2021.01.05 10| 3198.614 126219 14125100
AMZN   QUOTE ASK    2021.01.05 11| 3182.17  79024  9060300 
AMZN   QUOTE ASK    2021.01.05 12| 3192.846 66319  7049900 
AMZN   QUOTE ASK    2021.01.05 13| 3205.259 62269  7885300 
AMZN   QUOTE ASK    2021.01.05 14| 3209.761 80717  12378000
AMZN   QUOTE ASK    2021.01.05 15| 3186.998 105985 12370900
AMZN   QUOTE ASK    2021.01.05 16| 2938.977 486    48200   
AMZN   QUOTE ASK    2021.01.05 17| 3006.261 76     8200    
AMZN   QUOTE ASK    2021.01.05 18| 2481.

In [22]:
%%q --host $host --port $port --user $username --pass $password

/ time on server
.z.z

2024.11.26T20:09:01.041


In [23]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2024-11-26 20:09:01.048031
